In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [4]:
from ilqr.costs.finite_diff import FiniteDiffCost
from ilqr.mujoco_dynamics import MujocoDynamics
from ilqr.mujoco_controller import iLQR, RecedingHorizonController
from ilqr.utils.visualization import make_video_fn

from scipy.optimize import approx_fprime

import mujoco_py
from mujoco_py import MjViewer
import os

%load_ext autoreload
%autoreload 2

In [5]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    info = "converged" if converged else ("accepted" if accepted else "failed")
    final_state = xs[-1]
    print("iteration", iteration_count, info, J_opt)

In [6]:
xml_path = os.path.join('..', 'ilqr', 'xmls', 'swimmer.xml')
dynamics = MujocoDynamics(xml_path, frame_skip = 4, use_multiprocessing = True)
print(dynamics.dt)

Finished loading process Finished loading process5146 
Finished loading process5147 
5148Finished loading process 
Finished loading process5149 
5150Finished loading process
 5151Finished loading process
 5152
Finished loading process Finished loading process0.04
5153 
Finished loading process5154 
5155Finished loading process
 5156
Finished loading process 5157
Finished loading process Finished loading process5158 
5159Finished loading process 
5160
Finished loading process 5161


In [7]:
def l(x, u, i):
    action_cost = np.square(u).sum()
    vel_cost = 100 * ((x[5] - 3) ** 2)
    return action_cost + vel_cost

cost = FiniteDiffCost(l, lambda x, i: l(x, [0.0, 0.0], i), 10, 2, use_multiprocessing = True)

Finished loading process Finished loading process5162
 Finished loading process51635164

 Finished loading process 5165Finished loading process
 5166Finished loading process 
5167Finished loading process
 5168Finished loading process
 5169Finished loading process
 Finished loading process5170 
Finished loading process5171
 5172Finished loading process
 Finished loading process5173 
5174Finished loading process
 5175Finished loading process
 5176
Finished loading process 5177


In [8]:
N = 200
x0 = dynamics.get_state()
us_init = np.zeros((N, dynamics.action_size))
ilqr = iLQR(dynamics, cost, N)
mpc = RecedingHorizonController(x0, ilqr)

In [ ]:
t0 = time.time()
mpc_trajectory, controls = mpc.control(us_init, 20, step_size = 3, initial_n_iterations = 500, subsequent_n_iterations = 100, on_iteration = on_iteration)


iteration 0 accepted 168942.27900189822
iteration 1 accepted 168928.12448865589
iteration 2 accepted 168587.19339414127
iteration 3 accepted 166514.7297230467
iteration 4 accepted 165122.10325798584
iteration 5 accepted 162857.29220533028
iteration 6 accepted 161388.4060693477
iteration 7 accepted 160194.72996973328
iteration 8 accepted 159728.9655277769
iteration 9 accepted 158828.063098051
iteration 10 accepted 158508.84652724522
iteration 11 accepted 158040.2087872374
iteration 12 accepted 157613.60609784094
iteration 13 accepted 157318.83025620866
iteration 14 accepted 157013.6234826182
iteration 15 accepted 156807.82354603513
iteration 16 accepted 156565.49223499757
iteration 17 accepted 156286.89132872946
iteration 18 accepted 156029.44312702384
iteration 19 accepted 155756.60217526747
iteration 20 accepted 155487.0448030297
iteration 21 accepted 154656.75920756342
iteration 22 accepted 154182.38460880306
iteration 23 accepted 153961.93890366788
iteration 24 accepted 153782.98275

In [10]:
from mujoco_py import MjViewer
viewer = MjViewer(dynamics.sim)
dynamics.set_state(x0)
for control in controls:
    dynamics.step(control)
    viewer.render()

Creating window glfw


In [12]:
print(us.shape)

(187, 1)


In [13]:
make_video_fn(video)()